In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np

from afqinsight.datasets import AFQDataset

In [5]:
afq_dataset = AFQDataset(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [9]:
afq_dataset.drop_target_na()

In [10]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [11]:
dataset_size = len(afq_dataset.subjects)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)

full_dataset = afq_dataset.as_tensorflow_dataset()
full_dataset = full_dataset.shuffle(buffer_size=dataset_size, seed=0)

2021-12-01 09:35:02.422495: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [14]:
import tensorflow as tf

In [26]:
model = nn.cnn_lenet(input_shape=(100, 48), n_classes=1, output_activation=None, verbose=True)

pooling layers: 4
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 6)            870       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 50, 6)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 25, 2

In [23]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])

In [24]:
model.fit(train_dataset, epochs=50)

Epoch 1/50


ValueError: in user code:

    File "/opt/anaconda3/envs/afq-deep-learning/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/afq-deep-learning/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/afq-deep-learning/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/afq-deep-learning/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/envs/afq-deep-learning/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/envs/afq-deep-learning/lib/python3.8/